# 웹 크롤링 기본

* 크롤링 사이트: https://www.tripadvisor.co.kr/Restaurants-g294197-Seoul.html
* robots.txt:  https://www.tripadvisor.co.kr/robots.txt


### 크롤링 절차

1. 사이트의 html을 읽어들이기: requests.get(url) 사용
    
2. 텍스트 형태의 데이터를 html 태그별로 구분하여 파싱하기 : BeutifulSoup

3. 특정 태그값만 찾기 : findAll, find

4. 필요한 데이터값 정제하기

5. 데이터 저장하기

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# 크롤링할 웹 사이트 정보
# https://www.tripadvisor.co.kr/Restaurants-g294197-Seoul.html
base_url = 'https://www.tripadvisor.co.kr'
seoul_url = '/Restaurants-g294197-Seoul.html'

# get html
response = requests.get(base_url+seoul_url)
response

<Response [200]>

응답은 html 형태로 온다

In [3]:
response.text[:200]

'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml" lang="ko">\n<head>\n<meta http-equiv="content-type" content="text/html; charset=utf-8"/>\n<link rel=\'stylesheet\' type=\'text/css\' href=\'h'

In [4]:
# BeautifulSoup을 활용하여 데이터 파싱
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# 맛집 목록 가져오기
# <div class="wQjYiB7z"><span><a href="/Restaurant_Review-g294197-d8472275-Reviews-Choigozip_Hongdae-Seoul.html" class="_15_ydu6b" target="_blank">8<!-- -->. <!-- -->최고집 홍대점</a></span></div>
lists = soup.findAll('div', {'class' : 'wQjYiB7z'}) 
lists[:10]

# 이미지는 이렇게는 못받아옴. selenium 이용해야함
# lists = soup.findAll('div', {'class' : '_2Hb-Mt7l'}) 
# lists

[<div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g294197-d20941492-Reviews-Privilege_Bar-Seoul.html" target="_blank">1<!-- -->. <!-- -->프리빌리지 바</a></span></div>,
 <div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g294197-d21127141-Reviews-Cleo-Seoul.html" target="_blank">2<!-- -->. <!-- -->클레오</a></span></div>,
 <div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g294197-d21127142-Reviews-Blind_Spot-Seoul.html" target="_blank">3<!-- -->. <!-- -->블라인드 스팟</a></span></div>,
 <div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g294197-d14803574-Reviews-Flavors-Seoul.html" target="_blank">4<!-- -->. <!-- -->플레이버즈</a></span></div>,
 <div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g294197-d1978666-Reviews-Gusto_Taco-Seoul.html" target="_blank">5<!-- -->. <!-- -->구스토 타코</a></span></div>,
 <div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g294197-d10257

In [6]:
for item in lists[:5]:
    temp = str(item)
    print(temp)
    name = item.text
    print(name)

    # 광고 외의 식당이름만 뽑기
    # 지금은 식당 광고가 없어져서 필요없는 작업
    # name = item.text.split('. ')
    
    
    # if(len(name) == 2):
    #     print(name[0] + name[1])
    
    # 광고만 뽑기
    # if(len(name)<2):
    #     print(name[0])

    # 링크 가져오기
    hrefs = item.find('a')
    href = hrefs.get('href')
    full_url = base_url + href
    print(full_url + '\n')

<div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g294197-d20941492-Reviews-Privilege_Bar-Seoul.html" target="_blank">1<!-- -->. <!-- -->프리빌리지 바</a></span></div>
1. 프리빌리지 바
https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d20941492-Reviews-Privilege_Bar-Seoul.html

<div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g294197-d21127141-Reviews-Cleo-Seoul.html" target="_blank">2<!-- -->. <!-- -->클레오</a></span></div>
2. 클레오
https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d21127141-Reviews-Cleo-Seoul.html

<div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g294197-d21127142-Reviews-Blind_Spot-Seoul.html" target="_blank">3<!-- -->. <!-- -->블라인드 스팟</a></span></div>
3. 블라인드 스팟
https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d21127142-Reviews-Blind_Spot-Seoul.html

<div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g294197-d14803574-Reviews-Flavors-Seoul.html" target="_blank">4<!--

In [7]:
# 데이터 파싱하기
rest_id_list = []
rest_name_list = []
rest_url_list = []

for item in lists:
    # 링크 가져오기
    hrefs = item.find('a')
    href = hrefs.get('href')
    full_url = base_url + href

    # 순위와 이름 분리하기
    #[순위. 음식점 이름]
    #[광고음식점]    
    name = item.text.split('. ')

    if len(name) == 2:
        rest_id = name[0]
        rest_name = name[1]
        rest_url = full_url
        #데이터 프레임을 만들기 위한 리스트
        rest_id_list.append(rest_id)
        rest_name_list.append(rest_name)
        rest_url_list.append(rest_url)
#         print('[ID]' + rest_id + ' [NAME]' + rest_name + ' [URL]' + rest_url)
        
result = pd.DataFrame(data={'[ID]' : rest_id_list, '[NAME]' : rest_name_list, '[URL]' : rest_url_list})

result.head()

,[ID],[NAME],[URL]
0,1,프리빌리지 바,https://www.tripadvisor.co.kr/Restaurant_Revie...
1,2,클레오,https://www.tripadvisor.co.kr/Restaurant_Revie...
2,3,블라인드 스팟,https://www.tripadvisor.co.kr/Restaurant_Revie...
3,4,플레이버즈,https://www.tripadvisor.co.kr/Restaurant_Revie...
4,5,구스토 타코,https://www.tripadvisor.co.kr/Restaurant_Revie...


In [8]:
result[:10]

,[ID],[NAME],[URL]
0,1,프리빌리지 바,https://www.tripadvisor.co.kr/Restaurant_Revie...
1,2,클레오,https://www.tripadvisor.co.kr/Restaurant_Revie...
2,3,블라인드 스팟,https://www.tripadvisor.co.kr/Restaurant_Revie...
3,4,플레이버즈,https://www.tripadvisor.co.kr/Restaurant_Revie...
4,5,구스토 타코,https://www.tripadvisor.co.kr/Restaurant_Revie...
5,6,지화자,https://www.tripadvisor.co.kr/Restaurant_Revie...
6,7,853,https://www.tripadvisor.co.kr/Restaurant_Revie...
7,8,헴라갓,https://www.tripadvisor.co.kr/Restaurant_Revie...
8,9,장생건강원,https://www.tripadvisor.co.kr/Restaurant_Revie...
9,10,더그리핀바,https://www.tripadvisor.co.kr/Restaurant_Revie...


In [9]:
print(result[:10])

  [ID]   [NAME]                                              [URL]
0    1  프리빌리지 바  https://www.tripadvisor.co.kr/Restaurant_Revie...
1    2      클레오  https://www.tripadvisor.co.kr/Restaurant_Revie...
2    3  블라인드 스팟  https://www.tripadvisor.co.kr/Restaurant_Revie...
3    4    플레이버즈  https://www.tripadvisor.co.kr/Restaurant_Revie...
4    5   구스토 타코  https://www.tripadvisor.co.kr/Restaurant_Revie...
5    6      지화자  https://www.tripadvisor.co.kr/Restaurant_Revie...
6    7      853  https://www.tripadvisor.co.kr/Restaurant_Revie...
7    8      헴라갓  https://www.tripadvisor.co.kr/Restaurant_Revie...
8    9    장생건강원  https://www.tripadvisor.co.kr/Restaurant_Revie...
9   10    더그리핀바  https://www.tripadvisor.co.kr/Restaurant_Revie...


## (실습) 제주 맛집 이름, url 링크, 리뷰 수, 대표 리뷰 가져오기
크롤링할 웹 사이트 정보
'https://www.tripadvisor.co.kr/Restaurants-g297885-Jeju_Jeju_Island.html'


In [ ]:
jeju_url = '/Restaurants-g297885-Jeju_Jeju_Island.html'

response = requests.get(base_url + jeju_url)
response

In [15]:
# BeautifulSoup을 활용하여 데이터 파싱
soup = BeautifulSoup(response.text, 'html.parser')
# soup

In [72]:
# 각 데이터 받아올 리스트 생성
list_nm = []  # 제주 맛집 이름
list_url = [] # url 링크
list_rv = []  # 리뷰 수
list_rrv = [] # 대표 리뷰

In [73]:
lists = soup.findAll('div', {'class': 'wQjYiB7z'})
lists[:10]

[<div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g297885-d13137530-Reviews-Indian_Kitchen-Jeju_Jeju_Island.html" target="_blank">1<!-- -->. <!-- -->인디언키친</a></span></div>,
 <div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g297885-d8610476-Reviews-Myeongjin_Jeonbok-Jeju_Jeju_Island.html" target="_blank">2<!-- -->. <!-- -->명진전복</a></span></div>,
 <div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g297885-d9230927-Reviews-Ujin_Haejangguk-Jeju_Jeju_Island.html" target="_blank">3<!-- -->. <!-- -->우진해장국</a></span></div>,
 <div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g297885-d8647545-Reviews-Bluebird_By_Magpie-Jeju_Jeju_Island.html" target="_blank">4<!-- -->. <!-- -->블루버드 바이 맥파이</a></span></div>,
 <div class="wQjYiB7z"><span><a class="_15_ydu6b" href="/Restaurant_Review-g297885-d4033774-Reviews-Ollae_Guksu-Jeju_Jeju_Island.html" target="_blank">5<!-- -->. <!-- -->올래국수</a></span></div>,
 <div

In [74]:
# 맛집 이름, url 링크 저장
for item in lists:
  list_nm.append(item.text.split('. ')[1])
  list_url.append(base_url + item.find('a').get('href'))

print(list_nm[:10])
print(list_url[:10])

['인디언키친', '명진전복', '우진해장국', '블루버드 바이 맥파이', '올래국수', '마초 그릴', '고집돌우럭', '자매국수', '돈사돈', '삼다정']
['https://www.tripadvisor.co.kr/Restaurant_Review-g297885-d13137530-Reviews-Indian_Kitchen-Jeju_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Restaurant_Review-g297885-d8610476-Reviews-Myeongjin_Jeonbok-Jeju_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Restaurant_Review-g297885-d9230927-Reviews-Ujin_Haejangguk-Jeju_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Restaurant_Review-g297885-d8647545-Reviews-Bluebird_By_Magpie-Jeju_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Restaurant_Review-g297885-d4033774-Reviews-Ollae_Guksu-Jeju_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Restaurant_Review-g297885-d8492137-Reviews-Macho_Grill-Jeju_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Restaurant_Review-g297885-d10536441-Reviews-Gozipfish_topdong-Jeju_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Restaurant_Review-g297885-d9284562-Reviews-Jamae_Guksu-Jeju_Jeju_Island.html', '

In [76]:
# 리뷰 수 저장
import re

lists = soup.findAll('span', {'class': 'w726Ki5B'})

for item in lists:
  text = item.text
  list_rv.append(re.findall('\d+', item.text)[0])

list_rv[:10]

['435', '303', '195', '75', '258', '126', '96', '166', '118', '82']

In [77]:
# 대표 리뷰 저장
lists = soup.findAll('div', {'class': '_3bvktyA3'})

for item in lists:
  list_rrv.append([i.text for i in item.findAll('span', {'class': '_1OfeygW_'})])

list_rrv[:10]  

[['“저녁”', '“완전 굳!”'],
 ['“기대가 큰만큼 실망도 컸어요 ㅠㅠ”', '“명불허전”'],
 ['“저는 맛있게 먹었어요. 웨이팅이 긴데 기다리면”', '“전통적인 제주 음식”'],
 ['“공장에 못가서 맥주 사러 갔어요”', '“더 카페 더 비어”'],
 ['“고기 많이있어서 놀람”', '“고기국수”'],
 ['“멕시코음식 맛있는곳!”', '“제주에서 보기드문 멕시코 음식점.”'],
 ['“공항에서 가까워서 조아용”', '“공항에 매우 근접해서 맛 좋은 음식점”'],
 ['“저도 매번 여행 마치면서 마지막에 들리는 집으로”', '“제주도 가면 꼭 들리는 국수집”'],
 ['“맛있다!!”', '“고기가 두툼하고 젓갈비슷한 소스가 특히 마음에 들어요.”'],
 ['“조식부페 3일내내 똑같았음ㅠ”', '“조식”']]

In [78]:
result = pd.DataFrame(data={'[NAME]': list_nm, '[URL]': list_url, '[CNT_REVIEW]': list_rv, '[REVIEW]': list_rrv})

result.head()

,[NAME],[URL],[CNT_REVIEW],[REVIEW]
0,인디언키친,https://www.tripadvisor.co.kr/Restaurant_Revie...,435,"[“저녁”, “완전 굳!”]"
1,명진전복,https://www.tripadvisor.co.kr/Restaurant_Revie...,303,"[“기대가 큰만큼 실망도 컸어요 ㅠㅠ”, “명불허전”]"
2,우진해장국,https://www.tripadvisor.co.kr/Restaurant_Revie...,195,"[“저는 맛있게 먹었어요. 웨이팅이 긴데 기다리면”, “전통적인 제주 음식”]"
3,블루버드 바이 맥파이,https://www.tripadvisor.co.kr/Restaurant_Revie...,75,"[“공장에 못가서 맥주 사러 갔어요”, “더 카페 더 비어”]"
4,올래국수,https://www.tripadvisor.co.kr/Restaurant_Revie...,258,"[“고기 많이있어서 놀람”, “고기국수”]"
